In [ ]:
#Instalación de paquetes necesarios
!pip install gradio

#Importamos las librerias necesarias
import pandas as pd
import zipfile
import os
import numpy as np
import gradio as gr


#asociación de Collab con Kagle
  #carga de archivo json
from google.colab import files
#la siguiente linea la comento una vez cargado el token
#files.upload()

  #creamos una carpeta oculta en el sistema para alojar el json y le damos permisos
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#indicamos el dataset que queremos usar
!kaggle datasets download -d nehalbirla/vehicle-dataset-from-cardekho

# Descomprimir el archivo ZIP
with zipfile.ZipFile('vehicle-dataset-from-cardekho.zip', 'r') as zip_ref: zip_ref.extractall('cardekho_data')

# Verifica los archivos extraídos
os.listdir('cardekho_data')

# Cargar el dataset
df = pd.read_csv("cardekho_data/CAR DETAILS FROM CAR DEKHO.csv")


#---Preparación del dato---
#1º Añadir columnas adicionales (de forma aleatoria)
#colores de los vehículos
colores = ['rojo', 'azul', 'negro', 'blanco', 'gris', 'verde']
df['color'] = np.random.choice(colores, size=len(df))

#¿Incluye garantía?
df['garantia'] = np.random.choice(['sí', 'no'], size=len(df))

#Número de puertas
df['numero_puertas'] = np.random.choice([2, 3, 5], size=len(df))

#¿Incluye rueda de respuesto?
df['rueda_repuesto'] = np.random.choice(['sí', 'no'], size=len(df))

#¿Equipo de sonido compatible con Android Auto o Apple Car?
df['compatible_apple_android'] = np.random.choice(['sí', 'no'], size=len(df))

#2º Conversión del precio (de Rupias Indias a Euros)
# Tipo de cambio aproximado: 1 EUR = 90 INR
tipo_cambio = 90

#3º Cambiar nombre de valor "Petrol" por "Gasolina"
df['fuel'] = df['fuel'].replace('Petrol', 'Gasolina')

#4º Crear nueva columna con el precio en euros
df['precio_euros'] = (df['selling_price'] / tipo_cambio).round(2)

#5º Quedarme con las columnas que me interesan y ocultar las irrelevantes"df.head"
df[['name', 'precio_euros', 'year', 'seller_type', 'color', 'km_driven', 'fuel', 'transmission', 'owner', 'garantia', 'numero_puertas', 'compatible_apple_android', 'rueda_repuesto', ]].head()
df.drop(columns=['selling_price'], inplace=True)

#6º Limpiar del dataset de valores nulos
df.isnull().sum()

#---FIN de Preparación del dato---

# Almacenar mi propio CSV limpio sin la columna de indexación que crea Pandas
df.to_csv("archivolimpio.csv", index=False)

# Mostrar las primeras filas para verificar
#df.head()

# Función de recomendación
def recomendar_por_similitud(
    c1_year, c1_fuel, c1_color, c1_puertas, c1_trans, c1_seller_type, c1_km, c1_owner, c1_garantia, c1_apple, c1_repuesto,
    c2_year, c2_fuel, c2_color, c2_puertas, c2_trans, c2_seller_type, c2_km, c2_owner, c2_garantia, c2_apple, c2_repuesto,
    c3_year, c3_fuel, c3_color, c3_puertas, c3_trans, c3_seller_type, c3_km, c3_owner, c3_garantia, c3_apple, c3_repuesto,
    num_coches
):

    coches_usuario = []

    def crear_coche(year, fuel, color, puertas, trans, seller, km, owner, garantia, apple, repuesto):
        return {
            "year": year,
            "fuel": fuel,
            "color": color,
            "numero_puertas": puertas,
            "transmission": trans,
            "seller_type": seller,
            "km_driven": km,
            "owner": owner,
            "garantia": garantia,
            "compatible_apple_android": apple,
            "rueda_repuesto": repuesto
        }

    coches_usuario.append(crear_coche(c1_year, c1_fuel, c1_color, c1_puertas, c1_trans, c1_seller_type, c1_km, c1_owner, c1_garantia, c1_apple, c1_repuesto))
    if num_coches >= 2:
        coches_usuario.append(crear_coche(c2_year, c2_fuel, c2_color, c2_puertas, c2_trans, c2_seller_type, c2_km, c2_owner, c2_garantia, c2_apple, c2_repuesto))
    if num_coches == 3:
        coches_usuario.append(crear_coche(c3_year, c3_fuel, c3_color, c3_puertas, c3_trans, c3_seller_type, c3_km, c3_owner, c3_garantia, c3_apple, c3_repuesto))

    def calcular_similitud(coche_df):
        max_score = 0
        for coche_user in coches_usuario:
            score = 0
            for key in coche_user:
                if coche_df[key] == coche_user[key]:
                    score += 1
                elif key == "km_driven":
                    # Comparación por cercanía en km (±15%)
                    if abs(coche_df[key] - coche_user[key]) <= coche_user[key] * 0.15:
                        score += 1
                elif key == "year":
                    # Comparación por cercanía en año (±2 años)
                    if abs(coche_df[key] - coche_user[key]) <= 2:
                        score += 1
            max_score = max(max_score, score)
        return max_score

    df['similitud'] = df.apply(calcular_similitud, axis=1)
    recomendados = df.sort_values(by='similitud', ascending=False)

    # --- Generar PDF ---
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import PdfPages

    top = recomendados.head(10)
    pdf_path = "recomendados.pdf"

    with PdfPages(pdf_path) as pdf:
        fig, ax = plt.subplots(figsize=(10, len(top) * 0.5 + 1))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=top.values, colLabels=top.columns, loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1, 1.5)
        pdf.savefig(fig)
        plt.close()

    return top, pdf_path


# --- Interfaz Gradio ---
anio_min = df['year'].min()
anio_max = df['year'].max()
with gr.Blocks() as demo:
    gr.Markdown("# Desguaces Meco presenta un concepto innovador:")
    gr.Markdown("## Encuentra tu coche ideal en base a los que has tenido antes 🚗🏎️🚙")
    gr.Markdown("### Por favor, cuéntanos sobre tus coches anteriores:")

    num_coches = gr.Radio(choices=[1, 2, 3], label="¿Cuántos coches has tenido?", value=1)

    # Coche 1
    with gr.Group() as grupo1:
        gr.Markdown("#### Coche 1:")
        c1_year = gr.Slider(label="Año", minimum=anio_min, maximum=anio_max, step=1)
        c1_fuel = gr.Dropdown(df['fuel'].unique().tolist(), label="Combustible")
        c1_color = gr.Dropdown(df['color'].unique().tolist(), label="Color")
        c1_puertas = gr.Dropdown([2, 3, 5], label="Número de puertas")
        c1_trans = gr.Dropdown(df['transmission'].unique().tolist(), label="Transmisión")
        c1_seller_type = gr.Dropdown(df['seller_type'].unique().tolist(), label="Tipo de vendedor")
        c1_km_driven = gr.Slider(label="Kilometraje (en km)", minimum=0, maximum=807000, step=1000)
        c1_owner = gr.Dropdown(df['owner'].unique().tolist(), label="Número de propietarios")
        c1_garantia = gr.Dropdown(['sí', 'no'], label="¿Incluye garantía?")
        c1_compatible_apple_android = gr.Dropdown(['sí', 'no'], label="¿Compatible con Apple/Android?")
        c1_rueda_repuesto = gr.Dropdown(['sí', 'no'], label="¿Incluye rueda de repuesto?")

    # Coche 2
    with gr.Group(visible=False) as grupo2:
        gr.Markdown("#### Coche 2:")
        c2_year = gr.Slider(label="Año", minimum=anio_min, maximum=anio_max, step=1)
        c2_fuel = gr.Dropdown(df['fuel'].unique().tolist(), label="Combustible")
        c2_color = gr.Dropdown(df['color'].unique().tolist(), label="Color")
        c2_puertas = gr.Dropdown([2, 3, 5], label="Número de puertas")
        c2_trans = gr.Dropdown(df['transmission'].unique().tolist(), label="Transmisión")
        c2_seller_type = gr.Dropdown(df['seller_type'].unique().tolist(), label="Tipo de vendedor")
        c2_km_driven = gr.Slider(label="Kilometraje (en km)", minimum=0, maximum=807000, step=1000)
        c2_owner = gr.Dropdown(df['owner'].unique().tolist(), label="Número de propietarios")
        c2_garantia = gr.Dropdown(['sí', 'no'], label="¿Incluye garantía?")
        c2_compatible_apple_android = gr.Dropdown(['sí', 'no'], label="¿Compatible con Apple/Android?")
        c2_rueda_repuesto = gr.Dropdown(['sí', 'no'], label="¿Incluye rueda de repuesto?")

    # Coche 3
    with gr.Group(visible=False) as grupo3:
        gr.Markdown("#### Coche 3:")
        c3_year = gr.Slider(label="Año", minimum=anio_min, maximum=anio_max, step=1)
        c3_fuel = gr.Dropdown(df['fuel'].unique().tolist(), label="Combustible")
        c3_color = gr.Dropdown(df['color'].unique().tolist(), label="Color")
        c3_puertas = gr.Dropdown([2, 3, 5], label="Número de puertas")
        c3_trans = gr.Dropdown(df['transmission'].unique().tolist(), label="Transmisión")
        c3_seller_type = gr.Dropdown(df['seller_type'].unique().tolist(), label="Tipo de vendedor")
        c3_km_driven = gr.Slider(label="Kilometraje (en km)", minimum=0, maximum=807000, step=1000)
        c3_owner = gr.Dropdown(df['owner'].unique().tolist(), label="Número de propietarios")
        c3_garantia = gr.Dropdown(['sí', 'no'], label="¿Incluye garantía?")
        c3_compatible_apple_android = gr.Dropdown(['sí', 'no'], label="¿Compatible con Apple/Android?")
        c3_rueda_repuesto = gr.Dropdown(['sí', 'no'], label="¿Incluye rueda de repuesto?")

    def actualizar_visibilidad(n):
        return (
            gr.update(visible=True),
            gr.update(visible=n >= 2),
            gr.update(visible=n == 3),
        )

    num_coches.change(fn=actualizar_visibilidad, inputs=num_coches, outputs=[grupo1, grupo2, grupo3])

    boton = gr.Button("Buscar recomendaciones")
    salida = gr.Dataframe()
    pdf_salida = gr.File(label="Descargar PDF con resultados")

    boton.click(
        recomendar_por_similitud,
        inputs=[c1_year, c1_fuel, c1_color, c1_puertas, c1_trans, c1_seller_type, c1_km_driven, c1_owner, c1_garantia,
                c1_compatible_apple_android, c1_rueda_repuesto,
                c2_year, c2_fuel, c2_color, c2_puertas, c2_trans, c2_seller_type, c2_km_driven, c2_owner, c2_garantia,
                c2_compatible_apple_android, c2_rueda_repuesto,
                c3_year, c3_fuel, c3_color, c3_puertas, c3_trans, c3_seller_type, c3_km_driven, c3_owner, c3_garantia,
                c3_compatible_apple_android, c3_rueda_repuesto,
                num_coches],
        outputs=[salida, pdf_salida]
    )


# Ejecución de la interfaz
demo.launch(share=True, debug=True)


#print(df.columns)



Dataset URL: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho
License(s): DbCL-1.0
vehicle-dataset-from-cardekho.zip: Skipping, found more recently modified local copy (use --force to force download)
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ec34b9de7316fe8529.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
